In [1]:
import os
import re
import csv
import pandas as pd

In [2]:
def pripravi_imenik(ime_datoteke):
    imenik = os.path.dirname(ime_datoteke)
    if imenik:
        os.makedirs(imenik, exist_ok=True)

def zapisi_csv(slovarji, imena_polj, ime_datoteke):
    pripravi_imenik(ime_datoteke)
    with open(ime_datoteke, 'w', encoding='utf-8') as csv_datoteka:
        writer = csv.DictWriter(csv_datoteka, fieldnames=imena_polj)
        writer.writeheader()
        writer.writerows(slovarji)

In [3]:
vzorec_podatki = re.compile(
    r'<title>(?P<naslov>.*?)</title>.*?'
    r'<div class="kratek" itemprop="description">(?P<kratek_opis>.*?)</div>.*?'
    r'<div class="kontakt-opis"><h3>Kontaktni podatki</h3><p>\s*(?P<prodajalec>.*?)<br />',
    flags = re.DOTALL
)
vzorec_opis = re.compile(
    r'<h3>Dodaten opis</h3><div itemprop="disambiguatingDescription">(?P<opis>.*?)</div>',
    flags = re.DOTALL
)
vzorec_odvecno_html = re.compile(r'<.*?>')
vzorec_odvecno_vrstice = re.compile(r'\n')
vzorec_cena = re.compile(r'.ena: ?(?P<cena>.*?) ?EUR')
vzorec_lokacija = re.compile(r'<strong class="rdeca">(?P<lokacija>.*?)</strong>')
vzorec_velikost = re.compile(r'>, ?(?P<velikost>.*?) ?m2')
vzorec_leto = re.compile(r'l\. (?P<leto>....)')
vzorec_vrsta = re.compile(r'Prodaja..(?P<vrsta_1>.*?)[:,](?P<vrsta_2>.*?)[:,]')


In [4]:
podatki = []
opisi = []
count = 0
for filename in os.listdir("podatki\hise-mesto\posamezne"):
    if filename.endswith(".html"):
        with open("podatki/hise-mesto/posamezne/" + filename, encoding='utf-8') as datoteka:
            stran = datoteka.read()
            podatek = vzorec_podatki.search(stran).groupdict()
            opis = vzorec_opis.search(stran)
            if opis == None:
                count += 1
            else:
                opis = opis.groupdict()
                opis["opis"] = vzorec_odvecno_vrstice.sub(" ", vzorec_odvecno_html.sub("", opis["opis"]))
            opisi.append(opis)
            podatki.append(podatek)
for filename in os.listdir("podatki\hise-okolica\posamezne"):
    if filename.endswith(".html"):
        with open("podatki/hise-okolica/posamezne/" + filename, encoding='utf-8') as datoteka:
            stran = datoteka.read()
            podatek = vzorec_podatki.search(stran).groupdict()
            opis = vzorec_opis.search(stran)
            if opis == None:
                count += 1
            else:
                opis = opis.groupdict()
                opis["opis"] = vzorec_odvecno_vrstice.sub(" ", vzorec_odvecno_html.sub("", opis["opis"]))
            opisi.append(opis)
            podatki.append(podatek)
for filename in os.listdir("podatki\stanovanja-mesto\posamezne"):
    if filename.endswith(".html"):
        with open("podatki/stanovanja-mesto/posamezne/" + filename, encoding='utf-8') as datoteka:
            stran = datoteka.read()
            podatek = vzorec_podatki.search(stran).groupdict()
            opis = vzorec_opis.search(stran)
            if opis == None:
                count += 1
            else:
                opis = opis.groupdict()
                opis["opis"] = vzorec_odvecno_vrstice.sub(" ", vzorec_odvecno_html.sub("", opis["opis"]))
            opisi.append(opis)
            podatki.append(podatek)
for filename in os.listdir("podatki\stanovanja-okolica\posamezne"):
    if filename.endswith(".html"):
        with open("podatki/stanovanja-okolica/posamezne/" + filename, encoding='utf-8') as datoteka:
            stran = datoteka.read()
            podatek = vzorec_podatki.search(stran).groupdict()
            opis = vzorec_opis.search(stran)
            if opis == None:
                count += 1
            else:
                opis = opis.groupdict()
                opis["opis"] = vzorec_odvecno_vrstice.sub(" ", vzorec_odvecno_html.sub("", opis["opis"]))
            opisi.append(opis)
            podatki.append(podatek)
print(count)

48


In [5]:
lepi_podatki = []
count = 0
for podatek in podatki:

    index = podatki.index(podatek)
    lepi_podatki.append(dict())

    cena = vzorec_cena.search(podatek["kratek_opis"])
    lokacija = vzorec_lokacija.search(podatek["kratek_opis"])
    velikost = vzorec_velikost.search(podatek["kratek_opis"])
    leto = vzorec_leto.search(podatek["kratek_opis"])
    vrsta = vzorec_vrsta.search(podatek["naslov"])

    lepi_podatki[index]["cena"] = (cena.groupdict()["cena"] if cena != None else cena)
    lepi_podatki[index]["lokacija"] = (lokacija.groupdict()["lokacija"] if lokacija != None else lokacija)
    lepi_podatki[index]["velikost"] = (velikost.groupdict()["velikost"] if velikost != None else velikost)
    lepi_podatki[index]["leto"] = (leto.groupdict()["leto"] if leto != None else leto)
    lepi_podatki[index]["vrsta"] = (vrsta.groupdict()["vrsta_1"] if vrsta != None else vrsta)
    lepi_podatki[index]["podvrsta"] = (vrsta.groupdict()["vrsta_2"] if vrsta != None else vrsta)
    lepi_podatki[index]["prodajalec"] = podatek["prodajalec"]
    lepi_podatki[index]["opis"] = (opisi[index]["opis"] if opisi[index] != None else None)
    count += 1

In [6]:
zapisi_csv(lepi_podatki, ["cena", "lokacija", "velikost", "leto", "vrsta", "podvrsta", "prodajalec", "opis"], "osnovni_podatki.csv")

In [7]:
osnovni_podatki = pd.read_csv("osnovni_podatki.csv")
osnovni_podatki

,cena,lokacija,velikost,leto,vrsta,podvrsta,prodajalec,opis
0,"390.000,00",ČRNA VAS,"100,2",2016,Hiša,Dvojček,"MP PROJEKT, d.o.o., PE Ljubljana",Luksuzno in moderno opremljena hiša (enota dvo...
1,"890.000,00",DOBRUNJE,"758,2",2002,Hiša,Samostojna,ABC nepremičnine d.o.o.,"Moste – Dobrunje, parc.814m2 , stan.površine:7..."
2,"549.000,00","BIZOVIK, IDEALNO ZA VELIKE DRUŽINE ALI INVESTI...","327,6",1979,Hiša,Dvojček,"Stoja trade, d.o.o., Ljubljana","Prodamo prostoren dvojček na mirni, zeleni in ..."
3,"2.000.000,00","LJ. CENTER, TRSTENJAKOVA",450,2004,Hiša,Samostojna,Zasebna ponudba,NaN
4,"235.000,00",LJ. MOSTE,112,1925,Hiša,Vrstna,MEZANIN PRO nepremičnine d.o.o.,"Posredujemo pri prodaji manjša hiše, ki je kla..."
...,...,...,...,...,...,...,...,...
1795,"270.000,00","LOGATEC, CENTER","107,1",2021,Stanovanje,4-sobno,"VEBER Nepremičnine, Logatec, d.o.o.,","Novo štirisobno stanovanje na ključ, katerega ..."
1796,"1.700,00","LOGATEC, GRAD",230,2019,Stanovanje,5 in večsobno,"VEBER Nepremičnine, Logatec, d.o.o.,",Prodaja se luksuzno moderno stanovanje v REZID...
1797,"195.000,00","LOGATEC, SONČNI LOG",61,2005,Stanovanje,2-sobno,Zasebna ponudba,Prodamo pritlično stanovanje v naselju Sončni ...
1798,"cca 220.000,00","LOGATEC, SONČNI LOG","91,5",2005,Stanovanje,3,Elite nepremičnine d.o.o.,Prodamo prostorno in prijetno stanovanje v sos...
